# Clase 13 - Análisis Exploratorio de Datos

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Pablo Badilla**


## Objetivos de la Clase

![Etapas de un Proyecto de Ciencia de Datos](https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/13-EDA/metodologia.png)

Los datos son la materia prima para la contrucción de análisis estadísticos y modelos predictivos. Esto implica que las hipótesis y predicciones elaboradas por dichos modelos estarán directamente relacionados con la calidad de los datos. Por ende, es crítico asegurar un preprocesado y una buena examinación de los datasets a trabajar.

El contenido de esta cátedra, se centra en las técnicas esenciales para la exploración de datos usando las herramientas que hemos visto hasta el momento.


## Problema de Esta Clase: House Pricing

![House Pricing](https://storage.googleapis.com/kaggle-competitions/kaggle/5407/media/housesbanner.png)

El dataset **`house pricing`** consiste en 80 variables (79 variables explicativas más una variable objetivo) que describen aspectos fundamentales de hogares residenciales en la ciudad de _Ames, Iowa_.

La variable objetivo de este dataset es generar un modelo que prediga el precio final de cada hogar por medio de una regresión.

Es un problema "ultra conocido"y pueden encontrar su descripción aquí: https://www.kaggle.com/c/house-prices-advanced-regression-techniques


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm

# El conjunto a trabajar es el de entrenamiento
df = pd.read_csv(
    "https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/13-EDA//train.csv",
    index_col="Id",
)
df


In [ ]:
# Preprocesamiento

# Etapa 1: Convertir mes y año a Datetime
df["DateSold"] = pd.to_datetime(
    "1/" + df["MoSold"].astype(str) + "/" + df["YearBuilt"].astype(str), yearfirst=False
)

# Etapa 2: Generar cuartiles para los precios
df["SalePriceQCut"] = pd.qcut(
    df["SalePrice"], 4, ["Low", "Low-Mid", "Mid-High", "High"]
)

# Etapa 3: Convertir strings a categorías
df = pd.concat(
    [
        df.select_dtypes([], ["object"]),
        df.select_dtypes(["object"]).apply(pd.Series.astype, dtype="category"),
    ],
    axis=1,
)


In [ ]:
df = df.loc[
    :,
    [
        # Terreno 🏔️
        "LotArea",  # Area del terreno
        "LandSlope",  # Pendiente del terreno
        "Neighborhood",  # Barrio
        # Metadatos de la Vivienda 📆
        "BldgType",  # Tipo de vivienda
        "YearBuilt",  # Año de construcción
        "YearRemodAdd",  # Año de remodelación
        "Utilities",  # Agua, luz, etc...
        # Materiales 🧱
        "Foundation",  # Fundación de la vivienda
        "RoofMatl",  # Material del techo
        "RoofStyle",  # Estilo del techo
        "Exterior1st",  # Material del Exterior
        "ExterCond",  # Condición del material exterior
        # Interior de la casa 🏡
        "GrLivArea",  # Area habitable sobre el nivel del suelo.
        "1stFlrSF",  # Area primer piso
        "2ndFlrSF",  # Area segundo piso
        "FullBath",  # Baños completos
        "HalfBath",  # Medios baños, baños sin tina.
        "BedroomAbvGr",  # Piezas
        "KitchenAbvGr",  # Cocinas
        "KitchenQual",  # Calidad de la cocina
        # Sótano 🪨
        "TotalBsmtSF",  # Total sótano
        "BsmtCond",  # Condición del sótano
        # Garaje 🚗
        "GarageType",  # Tipo de garaje
        "GarageCars",  # Cantidad de autos por garaje
        # Piscina 🤽‍♂️
        "PoolArea",  # Area de la piscina
        "PoolQC",  # Calidad de la piscina
        # Calefacción y Aire 🌦️
        "Heating",  # Calefacción
        "HeatingQC",  # Calidad de la Calefacción
        "CentralAir",  # Aire Acondicionado Central
        # Calidad y Condición 🌟
        "OverallQual",  # Calidad general
        "OverallCond",  # Condición general actual
        # Datos de la venta  💵
        "DateSold",  # Mes y Año de venta
        "SaleType",  # Tipo de venta
        "SaleCondition",  # Condición de la vivienda en la venta
        "SalePrice",  # Precio de la venta
        "SalePriceQCut",  # Cuartiles del precio de la vivienda
    ],
]

numeric = df.select_dtypes(["int64"]).columns
categorical = df.select_dtypes(["category"]).columns


In [ ]:
df.info()


In [ ]:
df.head()


> **Actividad 📎**: ¿Qué hacemos ahora? - Diseñen en conjunto un plan para explorar los datos.


## Análisis Exploratorio de Datos

Un análisis exploratorio de datos (EDA, por sus siglas en inglés) es una técnica utilizada en la ciencia de datos para examinar y resumir las principales características de un conjunto de datos. **El objetivo principal del EDA es descubrir patrones, relaciones y tendencias en los datos, así como identificar cualquier anomalía o error que pueda estar presente.**


<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/13-EDA/eda_related.png' alt='Gráficos en análisis exploratorio de datos'/>

<div align='center'/>
Análisis exploratorio de datos. 
<br>
Fuente: <a href='https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15'>What is Exploratory Data Analysis?</a>
</center>


El EDA implica la utilización de herramientas estadísticas y gráficas para explorar los datos. Esto incluye la visualización de datos, el cálculo de medidas estadísticas como la media, la mediana y la desviación estándar, y la identificación de valores atípicos (o _outliers_) y errores.

En general, podemos resumir este proceso en estas 5 etapas, una del problema:

0. **Entender el problema con el que trabajaremos y obtener los datos.**

Y las otras respecto a los datos en si:

1. **Análisis Univariado**: En esta etapa se analiza cada variable de forma individual, utilizando técnicas estadísticas y gráficas para examinar su distribución, tendencias y valores atípicos.
2. **Análisis Bivariado y Multivariado**: En esta etapa se analizan las relaciones entre pares de variables, utilizando gráficos de dispersión y técnicas estadísticas para determinar la asociación entre ellas.
3. **Identificación y manejo de valores faltantes:** En esta etapa se identifican los valores faltantes en los datos y se decide cómo manejarlos para evitar sesgos en el análisis.
4. **Resumen y conclusiones:** En esta etapa se resumen los hallazgos clave del análisis exploratorio de datos y se extraen conclusiones sobre los datos y las relaciones entre las variables. Estas conclusiones pueden guiar el análisis posterior y la toma de decisiones.


### Repaso: Estadísticas de Resumen

Las estadísticas de resumen se refieren a un conjunto de medidas numéricas que resumen o describen las características clave de un conjunto de datos. Estas medidas proporcionan una forma rápida y sencilla de comprender los patrones y propiedades de los datos a través del análisis de su distribución, centralidad, variabilidad, entre otras.

Las estadísticas de resumen incluyen:

- Medidas de tendencia central (ver como se comportan los datos a través de valores centrales) como:

  1. Media (promedio): es la suma de todos los valores dividida por el número de observaciones.
  2. Mediana: es el valor que se encuentra en el centro de un conjunto de datos ordenados de menor a mayor.
  3. Moda: es el valor que ocurre con mayor frecuencia en un conjunto de datos.

- Dispersión (miden la variabilidad de los datos) como:

  1. Rango: es la diferencia entre el valor máximo y el valor mínimo en un conjunto de datos.
  2. Desviación estándar: mide la dispersión de los datos alrededor de la media.

- Percentiles y Cuartiles.

En el caso en que estemos analizando de forma multivariada, podemos usar:

- Correlación: mide el grado de asociación lineal entre dos variables.
- Tablas de contingencia: conteo entre ocurrencias de variables categóricas.


---


## 1. Análisis Univariado

El análisis univariado es el primer paso de un EDA y se enfoca en analizar cada variable de forma individual. Durante el análisis univariado, se utilizan diversas técnicas estadísticas y gráficas para explorar las características de una variable. Algunas de las técnicas comunes del análisis univariado incluyen:

1. Gráficos de frecuencia: son histogramas o gráficos de barras que muestran la frecuencia de cada valor en una variable.

2. Medidas de tendencia central: como la media, mediana y moda, que ayudan a resumir la ubicación central de los datos.

3. Medidas de dispersión: como la desviación estándar, varianza y rango, que miden cuánto se desvían los datos de la tendencia central.

4. Gráficos de caja: proporcionan una visión general de la distribución de los datos, incluyendo los cuartiles, el rango y los valores atípicos.

El objetivo final de esta etapa es comprender qué papel juega cada variable (de forma independiente) en la resolución del problema.


#### Análisis univariado numérico

En el siguiente ejemplo se mostrará cómo realizar un análisis de la variable numérica `SalePrice`.


In [ ]:
df.loc[:, ["SalePrice"]].describe().round(2)


> **Pregunta ❓**: ¿Cuál es el problema de la tabla anterior?


##### Histograma y Boxplot


Para mejorar el análisis anterior, usaremos un histograma más un boxplot o gráfico de caja.

1. Histograma: permite ver la distribución de los datos a través de barras que muestran la frecuencia de intervalos (bins) en una variable.
2. Boxplots: proporcionan una visión general de la distribución de los datos, incluyendo los cuartiles, el rango y los valores atípicos. El siguiente puntéo muestra el cómo leerlos:

- La caja muestra el primer y tercer cuartil.
- La mediana está mostrada en la división de la caja.
- El ancho de la caja es el rango intercuantílico (Q3 - Q1).
- Los brazos representan el rango intercuantílico \* +/- 1.5. Los valores fuera de este rango pueden ser considerados datos fuera de la norma / outliers.


In [ ]:
import plotly.express as px

px.histogram(
    df, 
    x="SalePrice", 
    title="Análisis de la variable SalePrice", 
    marginal="box"
)


Al finalizar, podemos concluir enseguida o juntar los resúmenes más adelante:

_Bajo estos gráficos podemos concluir que:_

- _La mayoría de las propiedades se concentran cerca del promedio de la distribución._
- _Existen algunas propiedades muy caras que hacen que la cola de la distribución sea muy larga hacia la derecha. Esto se confirma con el boxplot, en donde existen muchos puntos outliers._
- _Es relativamente uniforme a lo largo del dominio, salvo por algunas discontinudades, como por ejemplo en 170k a 180k._


### Caracterizar distribuciones

Existen algunas medidas útiles para caracterizar la distribución de una variable (aplican bien cuando las distribuciones son normales).


##### 1. **Skewness / Asimetría**

El skewness (o asimetría) de una distribución estadística es una medida de la falta de simetría en la forma de la distribución. En otras palabras, el skewness nos indica si la distribución está desplazada hacia la derecha o hacia la izquierda con respecto a su valor central.

Si una distribución es...

- Simétrica, su skewness es cero, lo que significa que la mitad de los datos están por encima del valor central y la otra mitad por debajo, y la distribución tiene un aspecto equilibrado.
- Asimétrica, su skewness es diferente de cero y la distribución tiene una forma sesgada hacia uno de los lados.

<div align='center'>
    <img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/13-EDA/skew.png' alt='Skewness' width=600/>
</div>


##### 2. **Curtosis**:

La curtosis mide el grado de concentración de los datos en torno a la media y la presencia de valores extremos (valores atípicos) en la distribución.

Una distribución con una curtosis...

1. Alta: tiene una concentración mayor de datos alrededor de la media y una cola más pronunciada, lo que indica que hay más valores extremos en la distribución.
2. Baja: tiene una concentración menor de datos alrededor de la media y una cola menos pronunciada, lo que indica que hay menos valores extremos en la distribución.

<div align='center'>
    <img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT1n1f_pfBRWXsET7Zk8UbRnzKLDPqsBs5ESL_uh9aKmlDcIpkO' alt='Kurtosis' width=600/>
</div>


In [ ]:
# skewness and kurtosis
print(f"Skewness: {df.loc[:, 'SalePrice'].skew().round(2)}")
print(f"Kurtosis: {df.loc[:, 'SalePrice'].kurt().round(2)}")


> **Pregunta ❓**: ¿Qué nos dicen estos valores de asimetría y curtosis de `SalePrice`?


In [ ]:
px.histogram(
    df, x="SalePrice", title="Análisis de la variable SalePrice", marginal="box"
)


### Distribuciones de Variables Numéricas

A continuación, generaremos histogramas que nos permitirán entender y visualizar la geometría de cada distribución.


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

n_cols = 4

fig = make_subplots(
    rows=len(numeric) // n_cols + 1, cols=n_cols, subplot_titles=numeric
)

for idx, col in enumerate(numeric):
    row_idx = idx // n_cols + 1
    col_idx = idx % n_cols + 1

    hist = go.Histogram(x=df.loc[:, col], name=col, histnorm="probability")
    fig.add_trace(hist, row=row_idx, col=col_idx)

fig.update_layout(
    height=800,
    title_text="Análisis Univariado de las Variables Numéricas",
    showlegend=False,
)
fig.show()


### Distribuciones de Variables Categóricas

Para las variables categóricas, se genera un conteo de las categorías a través de gráficos de barra.


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

n_cols = 4

fig = make_subplots(
    rows=len(categorical) // n_cols + 1, cols=n_cols, subplot_titles=categorical
)

for idx, col in enumerate(categorical):
    row_idx = idx // n_cols + 1
    col_idx = idx % n_cols + 1

    data = df.loc[:, col]
    hist = go.Histogram(x=data, name=col)
    fig.add_trace(hist, row=row_idx, col=col_idx)

fig.update_layout(
    height=1200,
    title_text="Análisis Univariado de las Variables Categoricas",
    showlegend=False,
)
fig.show()


Al observar las distribuciones, es importante buscar si existe variabilidad dentro de estas, pues por lo general, una variable con un único valor casi seguro, no aporta información a la dinámica de los datos.


## 2. Análisis Multivariado

El análisis bi y multivariado en un EDA implica el estudio simultáneo de dos o más variables en un conjunto de datos para determinar las relaciones entre ellas.

Las técnicas utilizadas en un análisis multivariado dependen de los tipos de variables involucradas:

- Si son de tipo numérico, se pueden utilizar:

  - Gráficos de dispersión y coordenadas paralelas.
  - Medidas de correlación.

- Si una de las variables es categórica y la otra numérica, se puede utilizar un gráfico de barras para comparar la distribución de la variable categórica en función de los valores de la variable numérica.

- Si son categóricas, se pueden utilizar:
  - Tablas de contingencia
  - Medidas de asociación, como el coeficiente de contingencia o el coeficiente de phi, para determinar la fuerza de la relación.


### Scatter Matrix

Una scatter matrix es una visualización en forma de matriz; en donde cada variable numérica se representa en un eje (vertical u horizontal) y cada celda en la matriz representa la relación entre dos variables numéricas.

La scatter matrix es útil para identificar patrones o relaciones complejas entre múltiples variables numéricas en un conjunto de datos y para identificar posibles valores atípicos o errores en los datos. Sin embargo, presenta debilidades en cuanto a la cantidad de variables que esta puede mostrar.


In [ ]:
variables_interes = [
    "GrLivArea",  # Area sobre el suelo
    "TotalBsmtSF",  # Area del sótano
    "FullBath",  # Número de baños
    "OverallQual",  # Calidad general
]


In [ ]:
fig = px.scatter_matrix(
    df.sort_values(by="SalePriceQCut", ascending=True),
    dimensions=variables_interes,
    height=1200,
    title="Análisis Bivariado",
    #color="SalePriceQCut",
).update_traces(diagonal_visible=False)

fig.show()


### Coordenadas paralelas

El gráfico de coordenadas paralelas es una visualización que permite comparar múltiples variables numéricas en un conjunto de datos. En este gráfico, cada variable se representa en un eje paralelo al resto de las variables, y las líneas que conectan los puntos representan las observaciones en el conjunto de datos.

El gráfico de coordenadas paralelas es útil para identificar patrones y relaciones complejas entre múltiples variables, y para identificar posibles valores atípicos o errores en los datos.
Una desventaja de este gráfico es que no muestra la distribución individual de cada variable, sino que muestra solamente la relación entre ellas.


In [ ]:
variables_interes = [
    "GrLivArea",  # Area sobre el suelo
    "TotalBsmtSF",  # Area del sótano
    "1stFlrSF",  # Area del primer piso
    "2ndFlrSF",  # Area del segundo piso
    "FullBath",  # Número de baños
    "YearBuilt",  # Año de construcción
    "OverallQual",  # Calidad
]

px.parallel_coordinates(df, dimensions=variables_interes, color="OverallQual")


La última fila de esta visualización entrega una idea de la relación entre las variables de interés. Dentro de las interacciones entre variables, se observa que 'GrLivArea' y 'TotalBsmtSf' se comportan de manera similar contra 'OverllQuall', esperandose cierta tendencia creciente en ambos casos.


### Análisis bivariado respecto a la variable dependiente

Cuando se crea un modelo predictivo, es común tener una variable dependiente o objetivo que se desea predecir. Para desarrollar un buen modelo, es importante analizar cómo se comportan las demás variables con respecto a dicha variable. El objetivo de análisis es esencial para identificar las variables más relevantes y comprender la naturaleza de la relación entre ellas y la variable objetivo.

Para esto, veremos 2 visualizaciones especiales:


#### Gráficos de violín

Un gráfico de violín permite sumarizar y observar características de un dataset. Este se comporta como un gráfico de cajas (boxplot), mostrando la mediana, el rango intercuantílico IQR (percentil 75 - percentil 25, o Q3 - Q1) y el rango 1.5 intercuantílico (Q3 +- 1.5 IQR). Sumado a lo anterior, se suma una estimación de la densidad por kernel a cada lado. Esto quiere decir, que zonas con mayor densidad, se verán como 'montes' horizontales.

A continuación veremos un par de ejemplos de esto:


##### OverallQual


In [ ]:
import plotly.express as px


fig = px.violin(
    df, 
    x="OverallQual", 
    y="SalePrice", 
    title="OverallQuall vs SalePrice",
).show()

fig = px.box(
    df, 
    x="OverallQual", 
    y="SalePrice", 
    title="OverallQuall vs SalePrice",
).show()

In [ ]:
fig = px.violin(
    df, x="OverallQual", 
    y="SalePrice", 
    title="OverallQuall vs SalePrice"
).show()

Se observa la variable `OverallQual` (categórica) y se compara con la variable de interés `SalePrice`. Para ello se usa un gráfico de categórias tipo violín y box (para adicionalmente comparar ambas visualizaciones)


##### LandSlope


In [ ]:
fig = px.violin(
    df, x="LandSlope", y="SalePrice", title="Violin plot LandSlope vs SalePrice"
)
fig.show()


Por otra parte, analizando las gráficas univariadas, se puede observar que para `LandSlope`, se tiene poca variablidad y no genera diferencias en distribución para 'SalePrice' en ninguna de sus categorias.


#### Scatterplots con Lineas de Tendencia

Otra técnica que se utiliza para analizar la relación entre dos variables en un modelo predictivo es el gráfico de dispersión con regresión lineal. Este gráfico permite visualizar la relación entre dos variables y estimar la magnitud y dirección de esa relación.


Por ejemplo, estudiemos la variable `GrLivArea`. Para esto, primero definimos una función para gráficar variables numéricas con respecto a `SalePrice`.


In [ ]:
fig = px.scatter(
    df,
    x="LotArea",
    y="SalePrice",
    title=f"LotArea vs SalePrice",
    trendline="ols",
)
fig.show()


En este caso, se puede observar una distrbución univariada bien definida y un comportamiento lineal aunque ruidoso. Esto hace que `GrLivArea` sea una variable de interés. De la misma manera, `1stFlrSF`, parece reflejar las mismas buenas características.


In [ ]:
fig = px.scatter(
    df,
    x="1stFlrSF",
    y="SalePrice",
    title=f"1stFlrSF vs SalePrice",
    trendline="ols",
)
fig.show()


En el caso de 'TotalBsmtSF' se tiene


In [ ]:
fig = px.scatter(
    df,
    x="TotalBsmtSF",
    y="SalePrice",
    title=f"TotalBsmtSF vs SalePrice",
    trendline="ols",
)
fig.show()


Una relación menos lineal con un poco más de ruido pero una buena distribución en e dataset. Esta variable puede ser de interés pero esto se puede estudiar a posteriori.

Finalmente para `PoolArea`, se tiene


In [ ]:
fig = px.scatter(
    df,
    x="PoolArea",
    y="SalePrice",
    title=f"PoolArea vs SalePrice",
    trendline="ols",
)
fig.show()


Se aprecia una distribución altamente concentrada en el 0 y poco relacionada con la variable a predecir.


#### Categorías Paralelas

La visualización de Categorías Paralelas permite comparar el comportamiento de variables categoricas a través de un gráfico 2d, en donde cada categoría es representada por una barra y conjuntos de datos similares se agrupan en cintas (_ribbons_).


In [ ]:
fig = px.parallel_categories(
    df.sample(1000).sort_values("SalePriceQCut", ascending=False),
    dimensions=[
        "OverallQual",
        "Exterior1st",
        "Utilities",
        "SalePriceQCut",
    ],
)
fig.show()


> **Ejercicio ✏️**: Explore el equivalente de Categorías Paralelas para variables numéricas ([Coordenadas paralelas](https://plotly.com/python/parallel-coordinates-plot/)) y grafique la relación entre variables numéricas y 'SalePrice'.


### Análisis Temporal

Este análisis nos permite ver como se comportan los valores de alguna variable a través del tiempo.


In [ ]:
df_temporal = (
    df.query("YearBuilt > 1920")
    .groupby("YearBuilt")
    .agg({"SalePrice": ["mean", "std"]})
    .sort_values("YearBuilt")
    .reset_index()
)

df_temporal.columns = ["YearBuilt", "SalePriceMean", "SalePriceStd"]

df_temporal


In [ ]:
px.line(
    df_temporal,
    y="SalePriceMean",
    x="YearBuilt",
)


In [ ]:
px.line(df_temporal, y="SalePriceMean", x="YearBuilt", error_y="SalePriceStd")


#### Graficar percentiles

También pueden graficar usando percentiles específicos:


In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)

    percentile_.__name__ = "percentile_%s" % n
    return percentile_


df_temporal_per = (
    df.query("YearBuilt > 1920")
    .groupby("YearBuilt")
    .agg({"SalePrice": [percentile(25), percentile(50), percentile(75)]})
    .sort_values("YearBuilt")
    .reset_index()
)
df_temporal_per.columns = ["YearBuilt", "SalePriceP25", "SalePriceP50", "SalePriceP75"]
df_temporal_per


> **Pregunta ❓**: ¿Qué podemos hacer en este caso para graficar cada columna con un color distinto?


In [ ]:
df_temporal_per = df_temporal_per.melt(
    id_vars=["YearBuilt"], value_name="SalePrice", var_name="Percentile"
)
df_temporal_per


In [ ]:
px.line(df_temporal_per, x="YearBuilt", y="SalePrice", color="Percentile")


#### Boxplot temporal

Una última opción es graficar usando boxplots.


In [ ]:
px.box(df.query("YearBuilt > 1920"), x="YearBuilt", y="SalePrice")


### Correlaciones entre variables


El análisis de correlaciones es una técnica que permite identificar patrones y relaciones entre las distintas variables de un dataset.

La correlación es una medida que cuantifica qué tanto dos variables se mueven juntas. Sus valores pueden ser:

- Positivo (cuando una variable aumenta, la otra también lo hace)
- Negativo (cuando una variable aumenta, la otra disminuye) 
- Cercana a cero (cuando las variables no están relacionadas).


La correlación se puede medir utilizando diferentes métodos, como el coeficiente de correlación de Pearson, el coeficiente de correlación de Spearman y el coeficiente de correlación de Kendall.
En pandas, podemos calcular la matriz de correlaciones por variables numericas usando el método `corr()`.

> **Pregunta ❓**: ¿Correlación implica causalidad?

Lectura interesante: https://www.jotdown.es/2016/06/correlacion-no-implica-causalidad/

In [ ]:
correlaciones = df.corr()
correlaciones

Y lo podemos graficas usando `px.imshow`


In [ ]:
px.imshow(
    correlaciones,
    aspect="16:9",
    title="Correlación entre Variables",
    height=800,
    zmin=-1,
    color_continuous_midpoint=0,
    zmax=1,
    color_continuous_scale=px.colors.sequential.Viridis,
)


Según el esquema de valores, se buscan los puntos más claros y más oscuros fuera de la diagonal. En primera instancia, las variables `TotalBsmtSF` y `1stFlrSF` parece bastante correlacionadas, lo mismo ocurre con la variables `GarageCars` y `GarageArea`, esto puede indicar multicolinearidad que implica información duplicada o relacionada de manera trivial en el dataset.


#### Correlaciones con respecto a la variable dependiente

Un caso especial del cálculo de las correlaciones es el de todas las variables con respecto a la dependiente: correlaciones cercanas al 1 o -1 pueden indicar que existe una fuerte relación entre alguna variable y la dependiente.

A continuación mostraremos las correlaciones de `SalePrice` con respecto a todas las otras.

In [ ]:
px.imshow(
    correlaciones.loc[["SalePrice"], :].sort_values("SalePrice", axis=1),
    zmin=-1,
    zmax=1,
    color_continuous_midpoint=0,
    color_continuous_scale=px.colors.sequential.Viridis,
    title="Correlación SalePrice con respecto a todas las variables"
)


Si bien, las correlaciones con `SalePrice` deben ser analizadas con más detenimiento, se pude ver claramente que `GrLivArea`, `TotalBsmtSF`, y `OverallQual` juegan un papel preponderante en el valor de la propiedad.


### Tablas de Contingencia

Para analizar valores categóricos (categórico vs categórico) existen herramientas especializadas una de ellas es por medio de tablas de dos tratamientos o de contingencia (2 way tables).

Estas permiten calcular el numero de ocurrencias de una variable para cada una de sus categorías en comparación con los valores de otra variable.

Por ejemplo, construyamos una tabla para analizar 'OverallQual' vs 'GarageCars':


In [ ]:
# Se construye la tabla
tabla = pd.crosstab(
    index=df.loc[:, "OverallQual"],
    columns=df.loc[:, "GarageCars"],
)
tabla


## 3. Identificación y manejo de valores faltantes

La tercera etapa consiste en buscar detectar y tratar los valores faltantes en los datos.

Los valores faltantes son aquellos datos que faltan en una o varias observaciones o variables del conjunto de datos. Esto puede ocurrir por varias razones, como errores de entrada de datos, datos faltantes o incompletos o simplemente porque no se recopiló esa información.

La identificación de los valores faltantes es el primer paso en este proceso. Esto implica la revisión del conjunto de datos para detectar los valores faltantes y determinar la cantidad y la ubicación de estos valores. Este paso puede llevarse a cabo mediante el uso de herramientas de visualización o estadísticas descriptivas.


Una vez que se han identificado los valores faltantes, el siguiente paso es el manejo de estos valores. Hay varias formas de tratar los valores faltantes, que incluyen:

- Eliminación de las observaciones o variables con valores faltantes.
- Imputación de los valores faltantes mediante la asignación de un valor estadístico, como la media o la mediana, o mediante técnicas más avanzadas como la imputación múltiple.
- Considerar la creación de nuevas variables para representar los valores faltantes.


En esta clase solo veremos el primer paso: identificar.

### Exploración de valores faltantes

Cuando los datos faltantes se encuentran en variables que no son de interés, se pueden obviar. Sin embargo, esto no es la regla general.


In [ ]:
df.isnull().sum()


Sin embargo, basta observar las columnas para comprender que tales variables si poseen valores faltantes


In [ ]:
var_missing = ["BsmtCond", "GarageType", "PoolQC"]

df[var_missing].head()


Por tal motivo es necesario realizar una exploración inicial de los datos faltantes en conjunción con los análisis de distribución iniciales.

**Ejercicios ✏️**

1. Estudie la distribución de los valores faltantes en las variables numéricas.

2. Considerando que para las variables categóricas las variables con valor 'nan' son consideradas como una nueva categoría. ¿Se ven afectados los análisis anteriores sobre sus distribuciones?


**Ejemplo**

Para estudiar en mayor profundidad la distribución de los valores faltantes, se procede a transformarlos en formato `np.nan`


In [ ]:
df.replace("nan", np.nan, inplace=True)


In [ ]:
px.bar(df.isnull().sum().sort_values())


### Nullity Matrix


En términos generales, los valores perdidos de este dataset se encuentran relativamente limpios pues están estandarizados con la categoría 'nan'.

Dado que sumarizar valores faltantes genera una estructura de datos, vale la pena explorarla visualmente, para facilitar tal tarea, existe la librería `missingno`


In [ ]:
# Instalamos el paquete usando conda o pip
import sys

!conda install --yes --prefix {sys.prefix} missingno
# !{sys.executable} -m pip install numpy # descomentar si se usa pip
import missingno as msno

Las visualizaciones de generadas por medio de esta librería pueden ser utilizadas para discutir el problema de valores faltantes y generan una estrategia para su tratamiento.


In [ ]:
df.isnull().sum().nlargest(10)


mediante la libreria `missingno` es posible ver el panorama completo de los valores faltanes en el dataset de manera sencilla


In [ ]:
import matplotlib.pyplot as plt
import missingno as msno

fig, ax = plt.subplots(figsize=[15, 10])

msno.matrix(df, ax=ax, sparkline=False)


Esta visualización muestra que exiten columnas practicamente sin información, según la agregación anterior, estas corresponden a `PoolQC`, `MiscFeature` y `Alley`.

Por medio de correlaciones entre valores faltantes, es posible obtener un análisis bivariado análogo al anteriormente generado. Para ello se puede utilizar un mapa de calor.


In [ ]:
fig, ax = plt.subplots(figsize=[15, 10])
msno.heatmap(df, ax=ax)

plt.show()


Este gráfico muestra correlaciones de nulidad entre pares de variables, estas varian desde -1 a 1, donde

- $-1$ significa que las variables son excluyentes, es decir, la aparición de una hace que la otra este ausente.
- $1$ corresponde inclusión, esto quiere decir, que la aparición de una hace que la otra aparezca. Valores cercanos a 0 (sin valor numérico en el gráfico) indican ausencia de relación de nulidad entre las variables.

En el gráfico recien generado, no se observen relaciones de nulidad negativa, por otra parte, existen variables fuertemente relacionadas en cuanto a su información como lo son 'MasVnrType' y 'MasVnrArea', el comportamiento general es que la información esta fuertemente relacionada (en el sentido de inclusión de información) o simplemente no lo está.

> **Ejercicio ✏️**

1. El gráfico de correlaciones de nulidad permite tener una idea de como se relaciona la información faltante en pares de variables. Para comparar más de dos variables es posible utilizar un _dendograma_. Utilice las 20 variables con mayor cantidad de valores faltanes visualice su dendograma por medio de `msno.dendogram`. Interprete los resultados.[_Hint_](https://github.com/ResidentMario/missingno)


> **Pregunta ❓**: ¿Qué hacemos con las columnas con datos faltantes?


---

## 4. `Pandas Profiling`

Pandas Profiling es una herramienta de perfilamiento automático de variables, el cuál dado un dataframe, genera un reporte en formato HTML.
El reporte contiene información detallada sobre cada variable más gráficos de interacciones y correlaciones entre estas. Según la documentación, los perfiles generados contienen:

- **Type inference**: detect the types of columns in a dataframe.
- **Essentials**: type, unique values, missing values
- **Quantile statistics** like minimum value, Q1, median, Q3, maximum, range, interquartile range
- **Descriptive statistics** like mean, mode, standard deviation, sum, median absolute deviation, coefficient of variation, kurtosis, skewness
- **Most frequent values**
- **Histogram**
- **Correlations** highlighting of highly correlated variables, Spearman, Pearson and Kendall matrices
- **Missing values matrix**, count, heatmap and dendrogram of missing values
- **Text analysis** learn about categories (Uppercase, Space), scripts (Latin, Cyrillic) and blocks (ASCII) of text data.
- **File and Image analysis** extract file sizes, creation dates and dimensions and scan for truncated images or those containing EXIF information.

Bien usada, es una herramienta muy poderosa que les permitirá acelerar mucho el Análisis Exploratorio de Datos.
Sin embargo, simpre estará la posibilidad de que tengan que programar algo "a mano" ya que el análisis entregado por el profiler no es suficiente.


In [ ]:
!pip install ipywidgets
!pip install pandas_profiling

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(
    df,
    title="Reporte Housing Pricing",
    explorative=True,
    vars={"num": {"low_categorical_threshold": 0}},  # resolver problema #954
)

# Problema 954:
# https://github.com/ydataai/pandas-profiling/issues/954

profile.to_file("report.html")
